### Decision Trees

Decision Trees can perform both classification and regression tasks, and even multioutput tasks.
They are also the fundamental components of Random Forests.

In [1]:
from __future__ import division, print_function, unicode_literals

import numpy as np
import os

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"

def image_path(fig_id):
    res_dir = os.path.join(PROJECT_ROOT_DIR, "img", "decision_trees")
    if not os.path.exists(res_dir):
        os.makedirs(res_dir)
    return os.path.join(res_dir, fig_id)

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width 
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2) 
tree_clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [3]:
from sklearn.tree import export_graphviz
export_graphviz(
    tree_clf,
    out_file=image_path("iris_tree.dot"), 
    feature_names=iris.feature_names[2:], 
    class_names=iris.target_names, 
    rounded=True,
    filled=True
)

To transform it to other format use:

$ dot -Tpng iris_tree.dot -o iris_tree.png

![source](img/decision_trees/iris_tree.png)

When making predicitions the model starts at the root node (depth 0, at the top): this node asks whether the flower’s petal length is smaller than 2.45 cm. If it is, then you move down to the root’s left child node (depth 1, left). In this case, it is a leaf node (i.e., it does not have any children nodes), so it does not ask any questions: you can simply look at the predicted class for that node and the Decision Tree predicts that your flower is an Iris-Setosa (class=setosa).

Node’s gini attribute measures its impurity: a node is “pure” (gini=0) if all training instances it applies to belong to the same class. For example, since the depth-1 left node applies only to Iris-Setosa training instances, it is pure and its gini score is 0.

Gini impurity:
\begin{equation}
G_i = 1 - \sum_{k=1}^{n}p_{i,k}^2
\end{equation}

Where $p_{i,k}$ is the ratio of class k instances among the training instances in the $i^{th}$ node.

In [4]:
tree_clf.predict_proba([[5, 1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [5]:
tree_clf.predict([[5, 1.5]])

array([1])

Decision Tree model is one of so called white-box models, because it us easy to interpret.

## CART algorithm

Scikit uses CART algorithm to train Decision Trees. The algorithm splits the training set in two subsets using single a feature k and a threshold $t_k$. It chooses k and $t_k$ by searching for a pair that produces the purest subsets(wighted by their size).

The cost function that the algorithm tries to minimize is:
\begin{equation}
J(k, t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}{m}G_{right}
\end{equation}

Where:
- $G_{left/right}$ measures the impurity of the left/right subset,
- $m_{left/right}$ is the number of instances in the left/right subset.

Once it has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets and so on, recursively. It stops recursing once it reaches the maximum depth (defined by the max_depth hyperparameter), or if it cannot find a split that will reduce impurity. 

The training algorithm compares all features (or less if max_features is set) on all samples at each node.

## Gini and Entropy

Gini is used by default in scikit but it can be changed to entropy impurity measure. Entropy is equal to 0 when all elements of set are from the same class.

Entropy measure:
\begin{equation*}
H_i = - \sum_{k=1, p_{i,k} \ne 0}^{n} p_{i,k}log(p_{i,k})
\end{equation*}

### Regularization Hyperparameters

If Decision Trees are left unconstrained, the tree structure will adapt itself to the training data, fitting it very closely, and most likely overfitting it and such models are called nonparametric models because the number of parameters is not determined prior to training.

A parametric model such as a linear model has a predetermined number of parameters, so its degree of freedom is limited, reducing the risk of overfitting.

For Decision Trees we should use regularization, and it is obtained by setting the maximum depth of the Decision Tree. Reducing max_depth will regularize the model and thus reduce the risk of overfitting.
Other hyperparameters that are possible to set in Decision Tree are:
- min_samples_split: the minimum number of samples a node must have before it can be split
- min_samples_leaf: the minimum number of samples a leaf node must have
- min_weight_fraction_leaf: same as min_samples_leaf but expressed as a fraction of the total number of weighted instances
- max_leaf_nodes: maximum number of leaf nodes
- max_features: maximum number of features that are evaluated for splitting at each node

Increasing min_* hyperparameters or reducing max_* hyperparameters will regularize the model.

# Regression

Decision Trees are also capable of performing regression tasks.

In [6]:
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

In [7]:
from sklearn.tree import DecisionTreeRegressor 
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=2,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [8]:
from sklearn.tree import export_graphviz
export_graphviz(
    tree_reg,
    out_file=image_path("reg_tree.dot"), 
    rounded=True,
    filled=True
)

![source](img/decision_trees/reg_tree.png)

In [9]:
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1]):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    plt.ylabel("$y$", fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

plt.figure(figsize=(16, 5))
plt.subplot(121)
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.subplot(122)
plot_regression_predictions(tree_reg2, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)

plt.show()

<Figure size 1600x500 with 2 Axes>

In this case Decision Tree are not predicting class name but a value.
The CART algorithm instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE.

CART cost function for regression:
\begin{equation*}
J(k, t_k) = \frac{m_{left}}{m}MSE_{left} + \frac{m_{right}}{m}MSE_{right}
\end{equation*}

where 
\begin{equation*}
MSE_{node} = \sum_{i \in node}(\hat{y} - y^{(i)})^2
\end{equation*}
and 
\begin{equation*}
\hat{y}_{node} = \frac{1}{m_{node}}\sum_{i \in node}y^{(i)}
\end{equation*}